In [1]:
import pypyodbc
from django.forms.models import model_to_dict
from django.utils.dateformat import format
from django.utils.timezone import make_aware
from django.db.models import F, Value, IntegerField, ExpressionWrapper, Q
from django.db.models.functions import Coalesce

import requests

#from .pypyodbcCRM import * 
import pypyodbc
import pandas as pd
import datetime
from datetime import date
import os
import subprocess
from dateutil.relativedelta import relativedelta

# Define the Components of the Connection String.
server = '192.168.0.21'
port = '4900'
database = 'ICAM'
username = 'admin'
password = 'Clip_SERENA'

# Variables dates
date_today = datetime.datetime.today()
formatted_date_today = date_today.strftime('%Y%m%d%H%M%S')
# date_limit = date_today + datetime.timedelta(days=42)
date_limit = date_today - relativedelta(years=1)
formatted_date_limit = date_limit.strftime('%Y%m%d')
print(formatted_date_limit)

# Create a connection object.
connection_object: pypyodbc.Connection = pypyodbc.connect('DRIVER={HFSQL}; \
                        Server Name =' + server + '; \
                        Server Port=' + port + '; \
                        DATABASE=' + database + '; \
                        UID=' + username + '; \
                        PWD=' + password)
                        #Trusted_Connection=yes;

# Create a Cursor Object, using the connection.
cursor_object: pypyodbc.Cursor = connection_object.cursor()

# Define the Select Query.
sql_select = "SELECT GAMME.GACLEUNIK, \
                CHARGRES.DAT as DATE_ORDO, \
                CHARGRES.HEUREDEB as HEURE_ORDO, \
                GAMME.DATEORDOOLD, \
                CLIENT.NOM as CLIENT_NOM, \
                AFFAIRE.NAF, \
                AFFAIRE.ETATAF, \
                GAMME.RANG, \
                GAMME.EN_PIECE, \
                GAMME.PHASE, \
                AFFAIRE.QTEAF, \
                AFFAIRE.QTEAFREST, \
                GAMME.GA_PREP, \
                GAMME.GA_NBH, \
                GAMME.GA_NBHR, \
                AFFAIRE.TYPEAF, \
                GAMME.OPFINIE, \
                GAMME.COFRAIS, \
                CHARGE.COSECT \
                FROM GAMME \
                LEFT JOIN AFFAIRE ON GAMME.NAF = AFFAIRE.NAF \
                LEFT JOIN ENGAM ON GAMME.ENCLEUNIK = ENGAM.ENCLEUNIK \
                LEFT JOIN CLIENT ON AFFAIRE.COCLI = CLIENT.COCLI \
                LEFT JOIN CHARGRES ON GAMME.GACLEUNIK = CHARGRES.GACLEUNIK \
                LEFT JOIN CHARGE ON GAMME.GACLEUNIK = CHARGE.GACLEUNIK \
                WHERE AFFAIRE.TYPEAF = 1 \
                AND GAMME.DATEORDOOLD > '"+formatted_date_limit+"' \
                AND GAMME.OPFINIE like 'O' "
                # AND CHARGE.DAT >= '"+formatted_date_today+"' \
                # AND CHARGE.DAT <= '"+formatted_date_limit+"'" # Voir pour enlever la borne inférieure en prod sur BDD CRM et mettre DATE_CHARGE IS NOT NULL
                # CHARGE.DAT as DATE_CHARGE, \
cursor_object.execute(sql_select)

# Define the column names.
#columns = [column[0] for column in cursor_object.statistics]
columns = [column[0] for column in cursor_object.description]

# Execute the Query.
# records = cursor_object.fetchmany(10)
records = cursor_object.fetchall()

print(records)

20230620
[(419839, None, None, datetime.datetime(2024, 5, 1, 17, 1), 'SNCF VOYAGEURS                          ', 40003, 'S', '001                                                         ', '04540230                      ', 100, 1.0, 0.0, 0.25, 0.0, 6.816667, '1', 'O', 'CTRL ', None), (419837, None, None, datetime.datetime(2024, 4, 29, 0, 46), 'SNCF VOYAGEURS                          ', 40003, 'S', '001                                                         ', '04540230                      ', 80, 1.0, 0.0, 0.25, 0.5, 1.466667, '1', 'O', 'MAJUS', None), (419836, None, None, datetime.datetime(2024, 5, 3, 14, 20), 'SNCF VOYAGEURS                          ', 40003, 'S', '001                                                         ', '04540230                      ', 70, 1.0, 0.0, 0.25, 0.0, 0.8, '1', 'O', 'CTRL ', None), (419835, None, None, datetime.datetime(2024, 5, 3, 3, 20), 'SNCF VOYAGEURS                          ', 40003, 'S', '001                                                   

#### one_row = crsr.fetchone()
type(one_row)